In [3]:
# Import packages and classes

import sys 
import os
import time
import pickle
sys.path.append('..')

from tqdm import tqdm

# Import packages
from src.fivecities.experiment import run_experiment, plot_results_timehorizons, plot_results_PTS, model_dict_default
from src.plotutils import print_table
from src.fivecities.reader import FiveCities

# Save images
save = True

In [4]:
# save results

def save_obj(obj, name ):
    with open('../results/fivecities/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


#save_obj(results_dict, f'results_dict_{int(time.time())}')

In [5]:
# load results (if desired)
def load_obj(name ):
    with open('../results/fivecities/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

filename = None
if filename:
    results_dict = load_obj(filename)

In [6]:
from src.model.baseline import RFRegressorLuPTS, KNRegressorLuPTS, MLPRegressorLuPTS, ModelWrapper
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Mute warnings from sklearn due to MLPRegressor not converging
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

model_dict_extended = model_dict_default.copy()
model_dict_extended['KNN'] = ModelWrapper(KNeighborsRegressor())
model_dict_extended['RF'] = ModelWrapper(RandomForestRegressor())
#model_dict_extended['MLP'] = ModelWrapper(MLPRegressor((10,), activation='tanh', solver ='lbfgs'))
model_dict_extended['RF tuned'] = RFRegressorLuPTS()
model_dict_extended['KNN tuned'] = KNRegressorLuPTS()
#model_dict_extended['MLP tuned'] = MLPRegressorLuPTS()


sample_size_list = [200]
list_res = []
for city in tqdm(FiveCities.city_list):
    res = run_experiment(city, 
                        sequence_length = 5,
                        timestep_list = [1],
                        n_list = sample_size_list,
                        model_dict=model_dict_extended)
    list_res.append(res)

  0%|          | 0/5 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Print out tables for LaTeX
for n_size in sample_size_list:
    
    print(f'Table for n={n_size}')
    df = print_table(list_res, n_size=n_size)

    filename = f'nonlinear_table_n{n_size}_{int(time.time())}.csv'
    df.to_csv('../results/fivecities/' + filename)
